In [ ]:

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
#import libraries
import warnings 
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
import math 
import re 
import pandas as pd
import os
import glob
from os.path import expanduser
import datetime
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)


In [ ]:
def color_negative_red(val):
    color = 'red' if val > 110 else 'black'
    return 'color: %s' % color


In [ ]:
def compute_rom(met,k,invT,nkeypoints):    
    #joint we want to evaluate in the current session
    jnt = getattr(k,met.ref_joint)
    jsess = jnt

    #direction wrt we want to evaluate the metric
    dirsess = met.ref_direction

    #joint child
    jskel = getattr(skel,met.ref_joint)

    cld = jskel.child
    childval = getattr(k,cld[0])
    child = childval

    #compute metric
    x=jsess[:,0]
    y=jsess[:,1]
    z=jsess[:,2]

    xchild=child[:,0]
    ychild=child[:,1]
    zchild=child[:,2]

    v1x = xchild-x
    v1y = ychild-y
    v1z = zchild-z
    v1=np.array([v1x,v1y,v1z])
    v1=np.transpose(v1)

    #plane over which we want to evaluate the metric
    planesess = met.ref_plane

    #project v1 on the right plane
    cosRom = []
    for i in range(0,nkeypoints):
        temp_ref = np.array([x[i],y[i],z[i],1])
        temp_child = np.array([xchild[i],ychild[i],zchild[i],1])

        transf_ref = np.inner(invT,temp_ref) #invT.dot(temp_ref)
        transf_child = np.inner(invT,temp_child) # invT.dot(temp_child)
        vprocess = transf_child-transf_ref
        vprocess = np.delete(vprocess,3)

        dist = np.dot(vprocess,np.transpose(planesess[i]))   
        vprocess = vprocess-dist*planesess[i]

        n1 = np.linalg.norm(vprocess)
        if(n1>0):
            vprocess = vprocess/n1

        n2 = np.linalg.norm(dirsess)
        if(n2>0):
            dirsess = dirsess/n2

        dotprod = np.dot(vprocess,np.transpose(dirsess))
        cosRom.append(dotprod)

    rom_value = np.arccos(cosRom)
    result = rom_value *(180/math.pi)

    return result


In [ ]:
def compute_ep(met,k,invT,nkeypoints,planesess,j1,j2):    
    #joint we want to evaluate in the current session
    jnt = getattr(k,met.ref_joint)
    #print jnt
    jsess = jnt
    x=jsess[:,0]
    y=jsess[:,1]
    z=jsess[:,2]

    v1=np.array([x,y,z])
    v1=np.transpose(v1)

    #project v1 on the right plane
    resultx = []
    resulty = []
    for i in range(0,nkeypoints):
        temp_jnt = np.array([x[i],y[i],z[i],1])   
        transf_jnt = np.inner(invT,temp_jnt) #invT.dot(temp_ref)
        vprocess = np.delete(transf_jnt,3)
        dist = np.dot(vprocess,np.transpose(planesess))      
        vprocess = vprocess-dist*planesess
        resultx.append(vprocess[j1])
        resulty.append(vprocess[j2])

    return [resultx,resulty]



## Personal data 

In [ ]:

#load file 
home = expanduser("~")
pth = home + '/.local/share/yarp/contexts/motionAnalyzer'
files = glob.glob(os.path.join(pth, '*.mat'))
lastfile = max(files, key=os.path.getctime)

#print personal data
i = [pos for pos, char in enumerate(lastfile) if char == "-"]
i1 = i[0]
i2 = i[1]
name = lastfile[i1+1:i2]
surname = ""
age = ""

personaldata = []
personaldata.append(name)
personaldata.append(surname)
personaldata.append(age)        
table = pd.DataFrame(personaldata) 
table.rename(index={0:"Name",1:"Surname",2:"Age"},  columns={0:"Patient"}, inplace=True)
table.style 

In [ ]:

data = []
ctime = []
filename = []
tagex = []
files.sort(key=os.path.getctime)
for fi in files:
    i = [pos for pos, char in enumerate(fi) if char == "-"]
    j = [pos for pos, char in enumerate(fi) if char == "_"]
    i1 = i[0]
    i2 = i[1]
    j1 = j[0]
    namei = fi[i1+1:i2]
    if namei == name:
        filename.append(fi)
        data.append(scipy.io.loadmat(fi))
        tagex.append(fi[i2+1:j1])
        ctime.append(os.path.getctime(fi)) 

In [ ]:
tagkeypoints = []
tagkeypoints.append('shoulderCenter')
tagkeypoints.append('head')
tagkeypoints.append('shoulderLeft')
tagkeypoints.append('elbowLeft')
tagkeypoints.append('handLeft')
tagkeypoints.append('shoulderRight')
tagkeypoints.append('elbowRight')
tagkeypoints.append('handRight')
tagkeypoints.append('hipLeft')
tagkeypoints.append('kneeLeft')
tagkeypoints.append('ankleLeft')
tagkeypoints.append('hipRight')
tagkeypoints.append('kneeRight')
tagkeypoints.append('ankleRight') 

In [ ]:
skeleton = namedtuple('skeleton', 
                      ['shoulderCenter','head','shoulderLeft','shoulderRight',
                      'elbowLeft','handLeft','elbowRight','handRight',
                      'hipLeft','hipRight','ankleLeft','ankleRight',
                      'kneeLeft','kneeRight'])

shoulderCenter = namedtuple('shoulderCenter', ['parent','child'])
head = namedtuple('head', ['parent','child'])
shoulderLeft = namedtuple('shoulderLeft', ['parent','child'])
shoulderRight = namedtuple('shoulderRight', ['parent','child'])
elbowLeft = namedtuple('elbowLeft', ['parent','child'])
handLeft = namedtuple('handLeft', ['parent','child'])
elbowRight = namedtuple('elbowRight', ['parent','child'])
handRight = namedtuple('handRight', ['parent','child'])
hipLeft = namedtuple('hipLeft', ['parent','child'])
hipRight = namedtuple('hipRight', ['parent','child'])
ankleLeft = namedtuple('ankleLeft', ['parent','child'])
ankleRight = namedtuple('ankleRight', ['parent','child'])
kneeLeft = namedtuple('kneeLeft', ['parent','child'])
kneeRight = namedtuple('kneeRight', ['parent','child'])

scSubst = shoulderCenter([''],['head','shoulderLeft','shoulderRight','hipLeft','hipRight'])
heSubst = head(['shoulderCenter'],[''])
slSubst = shoulderLeft(['shoulderCenter'],['elbowLeft'])
srSubst = shoulderRight(['shoulderCenter'],['elbowRight'])
elSubst = elbowLeft(['shoulderLeft'],['handLeft'])
hlSubst = handLeft(['elbowLeft'],[''])
erSubst = elbowRight(['shoulderRight'],['handRight'])
hrSubst = handRight(['elbowRight'],[''])
hilSubst = hipLeft(['shoulderCenter'],['kneeLeft'])
hirSubst = hipRight(['shoulderCenter'],['kneeRight'])
alSubst = ankleLeft(['kneeLeft'],[''])
arSubst = ankleRight(['kneeRight'],[''])
klSubst = kneeLeft(['hipLeft'],['ankleLeft'])
krSubst = kneeRight(['hipRight'],['ankleRight'])

skel = skeleton(scSubst,heSubst,slSubst,srSubst,elSubst,hlSubst,erSubst,
                  hrSubst,hilSubst,hirSubst,alSubst,arSubst,klSubst,krSubst)



In [ ]:
mini_rom = []
maxi_rom = []
tex_rom = []
ti_rom = []
tex_ep = []
ti_ep = []
result = []
xtraj = []
ytraj = []
ztraj = []
resultx_sagittal = []
resulty_sagittal = []
resultx_frontal = []
resulty_frontal = []
tottime = []
metric_tag1 = "ROM"
metric_tag2 = "EP"
for i in range(0,len(data)):

    datai = data[i]
    keypoints = namedtuple('keypoints', 
                      ['shoulderCenter','head','shoulderLeft','shoulderRight',
                      'elbowLeft','handLeft','elbowRight','handRight',
                      'hipLeft','hipRight','ankleLeft','ankleRight','kneeLeft','kneeRight'])
    time = datai['Time_samples']
    #print len(time)
    shoulderCenter = datai['Keypoints']['shoulderCenter'][0,0]
    #print len(shoulderCenter)
    head = datai['Keypoints']['head'][0,0]
    shoulderLeft = datai['Keypoints']['shoulderLeft'][0,0]
    shoulderRight = datai['Keypoints']['shoulderRight'][0,0]
    elbowLeft = datai['Keypoints']['elbowLeft'][0,0]
    handLeft = datai['Keypoints']['handLeft'][0,0]
    elbowRight = datai['Keypoints']['elbowRight'][0,0]
    handRight = datai['Keypoints']['handRight'][0,0]
    hipLeft = datai['Keypoints']['hipLeft'][0,0]
    hipRight = datai['Keypoints']['hipRight'][0,0]
    ankleLeft = datai['Keypoints']['ankleLeft'][0,0]
    ankleRight = datai['Keypoints']['ankleRight'][0,0]
    kneeLeft = datai['Keypoints']['kneeLeft'][0,0]
    kneeRight = datai['Keypoints']['kneeRight'][0,0]

    k = keypoints(shoulderCenter,head,shoulderLeft,shoulderRight,
        elbowLeft,handLeft,elbowRight,handRight,
        hipLeft,hipRight,ankleLeft,ankleRight,kneeLeft,kneeRight)   

    metric_tagi = ""
    if metric_tag1 in datai.keys():
        metric_tagi = metric_tag1
    if metric_tag2 in datai.keys():
        metric_tagi = metric_tag2

    metrici = namedtuple(metric_tagi, ['motion_type','ref_joint','ref_direction','ref_plane','max','min','tstart','tend'])
    motion_type = datai[metric_tagi]['motion_type'][0,0].tostring()
    motion_type = re.sub(r'[^\w]', '',motion_type)
    refjoint = datai[metric_tagi]['ref_joint'][0,0].tostring()
    refjoint = re.sub(r'[^\w]', '',refjoint)
    refdir = datai[metric_tagi]['ref_direction'][0,0]
    refplane = datai[metric_tagi]['ref_plane'][0,0]
    maxv = datai[metric_tagi]['max'][0,0]
    minv = datai[metric_tagi]['min'][0,0]
    tstart = datai[metric_tagi]['tstart'][0,0]
    tend = datai[metric_tagi]['tend'][0,0]
    met = metrici(motion_type,refjoint,refdir,refplane,maxv,minv,tstart,tend)

    nkeypoints = len(shoulderCenter)

    sagittal = k.shoulderLeft[0]-k.shoulderRight[0]
    sagittal = sagittal/np.linalg.norm(sagittal)
    transverse = k.shoulderCenter[0]-0.5*(k.hipLeft[0]+k.hipRight[0])
    transverse = transverse/np.linalg.norm(transverse)
    coronal = np.cross(sagittal,transverse)
    pSC = k.shoulderCenter[0]

    T = np.zeros((4, 4))
    T[0,0]=coronal[0]
    T[1,0]=coronal[1]
    T[2,0]=coronal[2]
    T[0,1]=sagittal[0]
    T[1,1]=sagittal[1]
    T[2,1]=sagittal[2]
    T[0,2]=transverse[0]
    T[1,2]=transverse[1]
    T[2,2]=transverse[2]
    T[0,3]=pSC[0]
    T[1,3]=pSC[1]
    T[2,3]=pSC[2]
    T[3,3]=1
    invT = np.linalg.inv(T)

    temp = []
    tempx = []
    tempy = []
    if metric_tagi == "ROM":
        temp = compute_rom(met,k,invT,nkeypoints)
        mini_rom.append(np.asscalar(min(temp)))
        maxi_rom.append(np.asscalar(max(temp)))
        tex_rom.append(np.asscalar(met.tend-met.tstart))
        ti_rom.append(datetime.date.fromtimestamp(ctime[i]).month)
        tottime.append(time)
        result.append(temp)

    if metric_tagi == "EP":
        plane_sagittal = np.array([0.0,1.0,0.0])
        plane_frontal = np.array([1.0,0.0,0.0])
        tempx,tempy = compute_ep(met,k,invT,nkeypoints,plane_sagittal,0,2)
        resultx_frontal.append(tempx)
        resulty_frontal.append(tempy)
        tempx,tempy = compute_ep(met,k,invT,nkeypoints,plane_frontal,1,2)           
        resultx_sagittal.append(tempx)
        resulty_sagittal.append(tempy)
        tex_ep.append(np.asscalar(met.tend-met.tstart))
        ti_ep.append(datetime.datetime.fromtimestamp(ctime[i]).month)
        tottime.append(time)

        jsess = getattr(k,met.ref_joint)
        x = jsess[:,0]
        y = jsess[:,1]
        z = jsess[:,2]
        xtraj.append(x)
        ytraj.append(y)
        ztraj.append(z)

    metric_tag_last = metric_tagi
    maxv_last = maxv
    lastmet = met


 
## Daily session report
The patient did the following exercise: 

In [ ]:

lastmet.motion_type 

 
on: 

In [ ]:

now = datetime.datetime.now()
print now 

 
The following shows the metric trend during the exercise:

In [ ]:
%matplotlib inline
%matplotlib inline
if metric_tag_last=="ROM":
    s = (len(tottime[-1]),1)
    time_list = [tup[0] for tup in tottime[-1]]
    res_list = [tup[0] for tup in result[-1]]
    ideal_list = [tup[0] for tup in maxv_last*np.ones(s)]
    trace1 = go.Scatter(
        x=time_list,y=res_list,
        mode='lines',
        line=dict(
            color='blue',
            width=3
        ),
        name='Real trend'
    )
    trace2 = go.Scatter(
        x=time_list,y=ideal_list,
        mode ='lines',
        line=dict(
            color='red',
            width=3
        ),
        name = 'Ideal trend'
    )
    data = [trace1,trace2]
    layout = dict(
        width=750,
        height=600,
        autosize=False,
        title='Range of Motion',
        font=dict(family='Courier New, monospace', size=22, color='black'),
        xaxis=dict(
            title='time [s]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='ROM [degrees]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
    fig = dict(data=data, layout=layout)
    iplot(fig)
if metric_tag_last=="EP":
    target = [0.0,1.0,2.0]
    trace1 = go.Scatter3d(
        x=xtraj[-1], y=ytraj[-1], z=ztraj[-1],
        mode = 'lines',
        line=dict(
            color='blue',
            width=3
        ),
        name = 'Trajectories'
    )
    trace2 = go.Scatter3d(
        x=[target[0]], y=[target[1]], z=[target[2]],
        mode = 'markers',
        marker=dict(
            color='red',
            size=5
        ),
        name = 'Target to reach'
    )
    data = [trace1, trace2]    
    layout = dict(
        #width=800,
        #height=700,
        #autosize=False,
        margin=dict(
            l=0,
            r=0,
            b=0
        ),
        title='End-point trajectories',
        font=dict(family='Courier New, monospace', size=22, color='black'),
        scene=dict(
            xaxis=dict(
                title='x [cm]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                ),
                tickfont=dict(
                    family='Courier New, monospace',
                    size=14,
                    color='#7f7f7f'
                ),
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            yaxis=dict(
                title='y [cm]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                ),
                tickfont=dict(
                    family='Courier New, monospace',
                    size=14,
                    color='#7f7f7f'
                ),
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            zaxis=dict(
                title='z [cm]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                ),
                tickfont=dict(
                    family='Courier New, monospace',
                    size=14,
                    color='#7f7f7f'
                ),
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            camera=dict(
                up=dict(
                    x=0,
                    y=0,
                    z=1
                ),
                eye=dict(
                    x=-1.7428,
                    y=1.0707,
                    z=0.7100,
                )
            ),
            aspectratio = dict( x=1, y=1, z=0.7 ),
            aspectmode = 'manual'
        ),
    )
    fig = dict(data=data, layout=layout)
    iplot(fig)      

 
## Patient progress

 
The exercises performed by the patient during the months under analysis are grouped as following:

In [ ]:
labels = ["reaching [EP]","abduction [ROM]","internal-rotation [ROM]","external-rotation [ROM]"]
values = [tagex.count("reaching"), tagex.count("internal"), tagex.count("external"), tagex.count("abduction")]
colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

trace = go.Pie(labels=labels, values=values,
               #hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)),
               hoverinfo="label+percent+value",
               hole=0.3
              )
layout = go.Layout(
        title="Performed exercises",
)
data = [trace]
fig = go.Figure(data=data,layout=layout)
iplot(fig) 

 
The trends of the patient metrics, grouped by month, are shown here: 

In [ ]:
if ti_rom:
    #group by month
    mprev = ti_rom[0]
    maxi_rom_mean = maxi_rom[0]
    mini_rom_mean = mini_rom[0]
    count = 1
    month = []
    maxi_rom_month = []
    mini_rom_month = []
    for i in range(1,len(ti_rom)):
        monthi = ti_rom[i]
        if monthi == mprev:
            maxi_rom_mean = maxi_rom_mean+maxi_rom[i]
            mini_rom_mean = mini_rom_mean+mini_rom[i]
            count = count+1
        else:
            maxi_rom_mean = maxi_rom_mean/count
            mini_rom_mean = mini_rom_mean/count
            maxi_rom_month.append(maxi_rom_mean)
            mini_rom_month.append(mini_rom_mean)
            month.append(mprev)
            count = 1
        if i == len(ti_rom)-1:
            maxi_rom_mean = maxi_rom_mean/count
            mini_rom_mean = mini_rom_mean/count
            mini_rom_month.append(mini_rom_mean)
            maxi_rom_month.append(maxi_rom_mean)
            month.append(mprev)
        mprev = monthi

    if len(month)==0:
        mini_rom_month.append(mini_rom_mean)
        maxi_rom_month.append(maxi_rom_mean)
        month.append(mprev)

    frame = pd.DataFrame({'t':month,'max':maxi_rom_month,'min':mini_rom_month})
    trace1 = go.Bar(
            x=frame['t'], 
            y=frame['max'],
            name='Maximum value reached',
            marker=dict(
                color='rgb(0,0,255)'
                )
            )
    trace2 = go.Bar(
            x=frame['t'], 
            y=frame['min'],
            name='Minimum value reached',
            marker=dict(
                color='rgb(255,0,0)'
                )
            )
    layout = go.Layout(
        title='Range of Motion Parameters',
        font=dict(family='Courier New, monospace', size=18, color='black'),
        xaxis=dict(
            title='Month',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            ),
            tickfont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='ROM [gradi]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            ),
            tickfont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        ),
        legend=dict(
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.1,
        bargroupgap=0.0
    )
    data=[trace1,trace2]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

if ti_ep:
    mprev = ti_ep[0]
    tex_ep_mean = tex_ep[0]
    count = 1
    month = []
    tex_ep_month = []
    for i in range(1,len(ti_ep)):
        monthi = ti_ep[i]
        if monthi == mprev:
            tex_ep_mean = tex_ep_mean+tex_ep[i]
            count = count+1
        else:
            tex_ep_mean = tex_ep_mean/count
            tex_ep_month.append(tex_ep_mean)
            month.append(mprev)
            count = 1
        if i == len(ti_ep)-1:
            tex_ep_mean = tex_ep_mean/count
            tex_ep_month.append(tex_ep_mean)
            month.append(mprev)
        mprev = monthi

    if len(month)==0:
        tex_ep_month.append(tex_ep_mean)
        month.append(mprev)

    frame = pd.DataFrame({'t':month,'tex':tex_ep_month})    
    trace1 = go.Bar(
            x=frame['t'], 
            y=frame['tex'],
            name='Tempo di esecuzione',
            marker=dict(
                color='rgb(100,255,200)'
                ),
            width=0.2
            )

    layout = go.Layout(
        title='End Point Parameters',
        font=dict(family='Courier New, monospace', size=18, color='black'),
        xaxis=dict(
            title='Month',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            ),
            tickfont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Execution time [s]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            ),
            tickfont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        ),
        legend=dict(
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.1,
        bargroupgap=0.0
    )
    data=[trace1]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig) 